In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from  pyspark.sql import SQLContext
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder \
             .master("local") \
             .appName("Word Count") \
             .config("spark.some.config.option", "some-value") \
            .getOrCreate()


In [3]:
dataSchema = StructType([StructField('Arrival_Time',LongType(),True),
                             StructField('Creation_Time',LongType(),True),
                             StructField('Device',StringType(),True),
                             StructField('Index',LongType(),True),
                             StructField('Model',StringType(),True),
                             StructField('User',StringType(),True),
                             StructField('gt',StringType(),True),
                             StructField('x',DoubleType(),True),
                             StructField('y',DoubleType(),True),
                             StructField('z',DoubleType(),True)
                        ])

In [4]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.json( '/home/henry/projects/Spark-The-Definitive-Guide/data/activity-data_small')


In [5]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [5]:
activityCounts = streaming.groupBy("gt").count()
activityQuery = activityCounts.writeStream.queryName("activty_counts").format("memory").outputMode("complete").start()
#activityQuery.awaitTermination()
spark.streams.active

In [6]:
spark.sql("select * from activty_counts").show()


+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|20905|
|       sit|24620|
|     stand|22770|
|      walk|26512|
|      bike|21594|
|stairsdown|18729|
|      null|20894|
+----------+-----+



In [9]:
#activityQuery.stop()
del(spark)